In [2]:
import pandas as pd
from metavision_core.event_io import EventsIterator
import numpy as np

from core.dsl.transformer.frequency_filter import FrequencyFilter
from core.event.subjects import PIPELINE_READY
import cv2
import os

# Modules to read, process and output events.
from core.dsl.transformer.mp4_to_sliding_events import Mp4ToSlidingEvents
from core.dsl.transformer.mp4_to_stacking_events import Mp4ToStackingEvents
from core.dsl.transformer.mp4_to_singular_events import Mp4ToSingularEvents
from core.dsl.transformer.frames_by_event_batches import EventBatchToFrames
from core.dsl.sink.event_writer import EventWriter
from core.dsl.transformer.frames_by_timestamps import FramesByTimestamps
from core.dsl.transformer.event_to_intensity_predictor import AsymptoticIntensityPredictor
from core.dsl.transformer.batch_throughput_limiter import BSync
from core.dsl.source.mp4_reader import Mp4Reader
from core.dsl.sink.window import Window
from core.dsl.source.events_reader import EventReader
from core.dsl.sink.mp4_writer import Mp4Writer
from core.dsl.transformer.mp4_to_greyscale import MP4ToGreyscale
from core.dsl.source.events_reader import EventReader
from core.dsl.transformer.naive_event_integrator import SpanningIntensityPredictor


def load_event_samples(directory, filetypes):
    absolute_path = os.path.abspath(directory)
    files = os.listdir(directory)
    return {file[:-4]: f'{absolute_path}/{file}' for file in files if file[-4:] in filetypes}

# Authentic events from event cameras.
event = load_event_samples('samples/events/', ['.raw', '.dat'])

# Conventional frame based video not derived from event cameras.
mp4 = load_event_samples('samples/mp4/', ['.mp4'])

# Conventional video deconstructed to synthetic events.
decon = load_event_samples('samples/decon/', ['.raw', '.dat'])

# Events reconstructed to regular video.
recon = load_event_samples('samples/recon/', ['.mp4'])




# mp4_in = Mp4Reader(mp4['formula1'])
# event_in = EventReader(event['spinner'], delta_t=1e4, max_duration=1e5)
#
# filter = FrequencyFilter(low_bound=0.0, high_bound=0.4)
# batch_frame = EventBatchToFrames()
# deconstructor = Mp4ToSingularEvents(threshold=0.1)
# reconstructor = AsymptoticIntensityPredictor(gaussian_filter_sigma=0.3, intensity_decay=0.001)
# window = Window('Mp4 roundtrip integrity test')
# recorder = Mp4Writer('samples/recon/driving-sample.mp4', fps=30)
#
# import plotly.express as px
# events = []
#
# event_in >> (lambda d,**k: events.append(d)) >> filter >> batch_frame >> window
#
# df = pd.DataFrame(columns=['x', 'y', 'p', 't'])
# for e in events:
#     e['t'] = e['t'][0]
#     df = pd.concat([df, pd.DataFrame(e[-10000:], columns=['x', 'y', 'p', 't'])])
#
#
# df['t'] = df['t'].astype(np.float64)
# fig = px.scatter_3d(df, x='x', y='t', z='y', color='t', template='plotly_white')
# fig.update_traces(marker_size= 5)
# fig.write_html('snapshots/plot-frame.html')

In [3]:
import pandas as pd
from metavision_core.event_io import EventsIterator
import numpy as np

from core.dsl.transformer.frequency_filter import FrequencyFilter
from core.event.subjects import PIPELINE_READY, KEYPRESS
import cv2
import os

# Modules to read, process and output events.
from core.dsl.transformer.mp4_to_sliding_events import Mp4ToSlidingEvents
from core.dsl.transformer.mp4_to_stacking_events import Mp4ToStackingEvents
from core.dsl.transformer.mp4_to_singular_events import Mp4ToSingularEvents
from core.dsl.transformer.frames_by_event_batches import EventBatchToFrames
from core.dsl.sink.event_writer import EventWriter
from core.dsl.transformer.frames_by_timestamps import FramesByTimestamps
from core.dsl.transformer.event_to_intensity_predictor import AsymptoticIntensityPredictor
from core.dsl.transformer.batch_throughput_limiter import BSync
from core.dsl.source.mp4_reader import Mp4Reader
from core.dsl.sink.window import Window
from core.dsl.source.events_reader import EventReader
from core.dsl.sink.mp4_writer import Mp4Writer
from core.dsl.transformer.mp4_to_greyscale import MP4ToGreyscale
from core.dsl.source.events_reader import EventReader
from core.dsl.transformer.naive_event_integrator import SpanningIntensityPredictor


def load_event_samples(directory, filetypes):
    absolute_path = os.path.abspath(directory)
    files = os.listdir(directory)
    return {file[:-4]: f'{absolute_path}/{file}' for file in files if file[-4:] in filetypes}

# Authentic events from event cameras.
event = load_event_samples('samples/events/', ['.raw', '.dat'])

# Conventional frame based video not derived from event cameras.
mp4 = load_event_samples('samples/mp4/', ['.mp4'])

# Conventional video deconstructed to synthetic events.
decon = load_event_samples('samples/decon/', ['.raw', '.dat'])

# Events reconstructed to regular video.
recon = load_event_samples('samples/recon/', ['.mp4'])



from core.dsl.source.events_reader import EventReader

event_in = EventReader(decon['formula1'], delta_t=int(1e5/(202/81)))

recorder = Mp4Writer('samples/recon/back6.mp4', fps=30)

reconstructor = AsymptoticIntensityPredictor(
    gaussian_filter_sigma=0.05,
    intensity_decay=0.0,
    intensity_impedance=1.0,
)

deconstructor = Mp4ToSingularEvents(threshold=0.07)

sync = BSync(batch_per_second=24)

batch_frame_generator = EventBatchToFrames()

mp4_out = Mp4Writer('samples/recon/formula1.mp4', fps=25)
window = Window('Mp4 roundtrip integrity test')
def save(**kwargs):
    print(kwargs)




def run():
    i = 0
    def do(data, **kwargs):
        nonlocal i
        i += 1
        if i == 100:
            cv2.imwrite(f'snapshots/snap{i}.png', data)
        if i > 100:
            raise Exception

    def imgsave(image, **kwargs):
        nonlocal i
        if i == 100:
            cv2.imwrite(f'snapshots/snap-mp4{i}.png', image)

    event_in >> reconstructor >> mp4_out

run()
mp4_in = Mp4Reader('samples/recon/formula1.mp4')
print(mp4_in.frame_cnt)

202


In [25]:
mp4_in = Mp4Reader('samples/recon/formula1.mp4')
print(mp4_in.frame_cnt)

464
